### Testing the construction of PySAL weights object from list of shapely objects

Author: David C. Folch <dfolch@gmail.com>

Based on Serge Rey's weights_from_geojson.ipynb

In [1]:
import pysal as ps
from shapely.geometry import MultiPoint
import numpy as np
from scipy.spatial import Voronoi

In [2]:
class PolygonCollection:
    def __init__(self, polygons, bbox=None):
        """

        Parameters
        ==========
        polygons: dict
                  key is polygon Id, value is PySAL Polygon object
        bbox: list (optional)
              [left, lower, right, upper]

        Notes
        =====
        bbox is supported in geojson specification at both the feature and feature collection level. However, not all geojson writers generate the bbox at the feature collection level. 
        In those cases, the bbox property will be set on initial access.

        """
              
        self.type=ps.cg.Polygon
        self.n = len(polygons)
        self.polygons = polygons
        if bbox is None:
            self._bbox = None
        else:
            self._bbox = bbox
            
    @property
    def bbox(self):
        bboxes = np.array([self.polygons[p].bbox for p in self.polygons])
        mins = bboxes.min(axis=0)
        maxs = bboxes.max(axis=0)
        self._bbox = [ mins[0], mins[1], maxs[2], maxs[3] ]
        return self._bbox
        
    
    def __getitem__(self, index):
        return self.polygons[index]

Some code to get finite voronoi polygons (ignores the vertices at infinity, which works for my use case). Basically it's a way to gin up some random connected shapely objects.

In [3]:
def get_voronoi(points):
    vor = Voronoi(points)
    vor_polys = []
    for region in vor.regions:
        if region:
            if -1 in region:
                region = region[:] # so we don't change the original voronoi output
                region.remove(-1)  # there can only be one -1 in the index list
            vor_polys.append(MultiPoint(vor.vertices[region]).convex_hull)
    return vor_polys

In [4]:
def shapely2w(polys, wttype):
    '''
    Parameters:
    ----------
    polys:  list
            List of shapely polygons
    wttype: string
            "rook" or "queen"

    Returns
    -------
    pysal weights object

    '''
    if wttype.upper() == 'QUEEN':
        wttype = 1
    elif wttype.upper() == 'ROOK':
        wttype = 2
    else:
        raise Exception, "unknown weight type"

    polys_pysal = []
    poly_ids = []
    poly_index = 0
    for feature in polys:
        polys_pysal.append(ps.cg.asShape(feature))
        poly_ids.append(poly_index)
        poly_index +=1
    pcollection = PolygonCollection(dict(zip(poly_ids, polys_pysal)))
    neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=wttype).w
    return ps.W(neighbors)

In [5]:
np.random.seed(999)
pnts = np.random.random((100, 2))
# get some random shapely polygons
vor_polys_shapely = get_voronoi(pnts)
# build a W object from the polygons
w = shapely2w(vor_polys_shapely, 'queen')

In [6]:
w.n

100